### 선별된 파일 목록을 불러온 후 Train-Validation으로 분할하고 이미지 파일 복사, 라벨 파일 생성

원천 데이터에서 선별한 데이터를 Training 데이터셋과 Validation 데이터셋으로 분할할 것입니다.

이후 YOLOv5 전이학습에서 요구하는 포맷에 맞춰 이미지 파일을 복사시키고, 라벨 파일도 새롭게 생성할 것입니다.

작업이 끝나고 나면 최종적으로 아래와 같은 폴더 구조를 가지게 될 것입니다.

>&nbsp;&nbsp;<br>
>D드라이브<br>
>&nbsp;&nbsp;└ pgu_traffic_dataset<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ sources → 원천데이터가 담긴 폴더<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ images<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ annotations<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ selected → Yolov5에 맞춰 우리가 생성할 폴더<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ training<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ images<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ labels<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ validation<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ images<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ labels<br>
>&nbsp;&nbsp;

In [ ]:
import os
import cv2
import json
import shutil
import random
import pandas as pd
from tqdm import tqdm

저장한 선별 파일 목록 로드 후 Training 데이터셋과 Validation 데이터셋으로 분할합니다.

In [ ]:
#저장한 선별 파일 목록(json) 로드
final_selected_ann_file_list = json.loads(open("최종 선별된 파일 목록.json", "r").read())
len(final_selected_ann_file_list)

In [ ]:
#df로 변환
df = pd.DataFrame(data={ "ann파일명" : final_selected_ann_file_list})
df

In [ ]:
#train validation set 분할
train = df.sample(frac=0.8, random_state=1234)
train

In [ ]:
validation = df.drop(train.index)
validation

파일을 복사, 저장할 폴더를 생성합니다. 파일 탐색기에서 수작업으로해도 되고 코드로 해도 됩니다.

1. D드라이브 'pgu_traffic_dataset' 폴더 하위에 'selected' 폴더 생성

2. 'selected' 하위에 'training'과 'validation' 폴더 생성

3. 'training' 하위에 'images'와 'labels' 폴더 생성

4. 'validation' 하위에 'images'와 'labels' 폴더 생성

작업이 끝나면 아래와 같은 폴더 구조가 만들어졌는지 확인합니다.

>&nbsp;&nbsp;<br>
>D드라이브<br>
>&nbsp;&nbsp;└ pgu_traffic_dataset<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ sources → 원천데이터가 담긴 폴더<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ images<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ annotations<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ selected → Yolov5에 맞춰 우리가 생성할 폴더<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ training<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ images<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ labels<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ validation<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ images<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ labels<br>
>&nbsp;&nbsp;

In [ ]:
src_path = "D:/pgu_traffic_dataset/sources"
dst_path = "D:/pgu_traffic_dataset/selected"

#train, validation 이미지 파일 복사 및 라벨 생성
# target_dir = "training"
# target_df = train.copy()

target_dir = "validation"
target_df = validation.copy()
for ann_file_name in tqdm(target_df["ann파일명"].tolist()):
    #이미지 파일을 selected 폴더에 있는 training > images 폴더로 복사한다.
    src_img_path = "{}/images/{}.png".format(src_path, ann_file_name.split(".")[0])
    dst_img_path = "{}/{}/images/{}.png".format(dst_path, target_dir, ann_file_name.split(".")[0])
    shutil.copy(src_img_path, dst_img_path) #이미지 파일 복사

    #라벨 파일을 연다음 Yolov5에 맞춰 파일을 생성하고 training > labels 폴더로 복사한다.
    ann_data = json.loads(open("{}/annotations/{}".format(src_path, ann_file_name), "r").read())
    
    #바운딩 박스를 0~1 값으로 rescaling을 하기 위해 이미지의 너비/높이 값을 가져와 저장한다.
    iw = ann_data["images"]["width"]
    ih = ann_data["images"]["height"]
    label_text = ""
    for ann in ann_data["annotations"]:
        #Yolo 모델은 바운딩 박스 라벨링 포맷을 centerx, centery, width, height로 하고 있다.
        #또한 0~1 사이 값으로 표준화된 수치로 넣어줘야 하므로, 거기에 맞춰 바운딩 박스 정보를 편집한다.
        x = int(ann["bbox"][0])
        y = int(ann["bbox"][1])
        w = int(ann["bbox"][2])
        h = int(ann["bbox"][3])
        cx = x + int(w/2)
        cy = y + int(h/2)

        #0~1 사이 값으로 표준화한다.
        cx = cx/iw
        cy = cy/ih
        w = w/iw
        h = h/ih

        if ann["category_id"] == 2:
            label_text += "{}\t{}\t{}\t{}\t{}\n".format("0", cx, cy, w, h) #2번 클래스는 class no를 0으로 치환한다.
        elif ann["category_id"] == 3:
            label_text += "{}\t{}\t{}\t{}\t{}\n".format("1", cx, cy, w, h) #3번 클래스는 class no를 1로 치환한다.
    label_text = label_text[:-1] #문자열 마지막에 줄바꿈 기호(\n)이 있으므로 제거해 준다.

    #Yolo 포맷에 맞춰 완성된 라벨 내용을 파일로 기록하여 저장한다.
    dst_label_path = "{}/{}/labels/{}.txt".format(dst_path, target_dir, ann_file_name.split(".")[0])
    open(dst_label_path, "w", encoding="utf-8").write(label_text)

이미지를 랜덤하게 가져와 라벨링 정보가 올바른지 확인해 봅니다.

그전에 먼저, 지금까지 우리가 만든 폴더의 구조를 다시 이해해 봅시다.


>&nbsp;&nbsp;<br>
>D드라이브<br>
>&nbsp;&nbsp;└ pgu_traffic_dataset<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ sources<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ images<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ annotations<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ selected<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ training<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ images<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ labels<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ validation<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ images<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ labels<br>
>&nbsp;&nbsp;

In [ ]:
img_path = "D:/pgu_traffic_dataset/selected/training/images"
label_path = "D:/pgu_traffic_dataset/selected/training/labels"

label_file_list = os.listdir(label_path)
random_no_list = [random.randrange(0, len(label_file_list)) for _ in range(0, 30)] # 랜덤값을 인덱스하여 총 30장의 이미지만 무작위로 뽑아올 수 있게 한다.

for idx in random_no_list:
    label_str = open("{}/{}".format(label_path, label_file_list[idx]), "r").read()

    #openCV를 활용해 이미지 파일을 띄운 다음 라벨링 정보를 반영해 바운딩박스를 그려봄
    src = cv2.imread("{}/{}.png".format(img_path, label_file_list[idx].split(".")[0]))
    dst = src.copy()
    for bbox in label_str.split("\n"):
        bbox = [float(x) for x in bbox.split("\t")]
        cx = int(bbox[1]*src.shape[1])
        cy = int(bbox[2]*src.shape[0])
        w = int(bbox[3]*src.shape[1])
        h = int(bbox[4]*src.shape[0])
        x = int(cx - (w/2))
        y = int(cy - (h/2))
        c_no = str(int(bbox[0]))
        dst = cv2.rectangle(dst, (x, y, w, h), (0, 255, 0), 1)
        dst = cv2.putText(dst, str(c_no), (x, y), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 0), 1)
    cv2.imshow("dst", dst)
    if cv2.waitKey() == 27:
        break
    else:
        continue
cv2.destroyAllWindows()

백그라운드 이미지(라벨이 없는 이미지)를 5~10% 넣어두면 모델 정확도를 높이고 오탐지(False Positive)를 줄일 수 있습니다.

제외된 이미지들 중 일부(트레이닝 데이터 개수 4807개의 10%인 480개)를 랜덤하게 선택한 후 training > images 폴더에 복사해 넣습니다.

In [ ]:
#저장한 백그라운드 파일 목록(json) 로드
background_ann_file_list = json.loads(open("백그라운드 파일 목록.json", "r").read())
len(background_ann_file_list)

In [ ]:
#df로 변환
df = pd.DataFrame(data={ "ann파일명" : background_ann_file_list})
df

In [ ]:
#train test set 분할
backgrounds = df.sample(frac=0.36, random_state=1234) #1335개 중 36%, 즉 481개만 랜덤으로 추려낸다.
backgrounds

In [ ]:
#백그라운드 이미지 파일을 selected 폴더에 있는 training > images 폴더로 복사한다. 라벨 파일 복사는 불필요하다.
target_dir = "training"
for ann_file_name in tqdm(backgrounds["ann파일명"].tolist()):
    src_img_path = "{}/images/{}.png".format(src_path, ann_file_name.split(".")[0])
    dst_img_path = "{}/{}/images/{}.png".format(dst_path, target_dir, ann_file_name.split(".")[0])
    shutil.copy(src_img_path, dst_img_path) #이미지 파일 복사